# Match CAM and Save Image - Multiple Folder

### Import Package

In [1]:
## Package
from PIL import Image
from datetime import datetime
import random, os, sys, glob
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
module_path = '/home/sasdemo01'
if module_path not in sys.path:
    sys.path.append(module_path)
import util
from util.preprocess import *

### Setting Parameters

In [15]:
## Data name
Father_folder = "0212"

## CAM File (.bmp)
cam = Image.open('/home/sasdemo01/data/AVI/S1/CAM/5143b69c01-uts+1_CompSideUnit.bmp')

## Training Folder (Folder)
Img_path = "/home/sasdemo01/data/AVI/S1/Testing/"+Father_folder +"/"

## Training Converted (Folder)
Converted_path = "/home/sasdemo01/data/AVI/S1/Testing_Convert/"+Father_folder+"/"

## Unit Position (.csv)
mapcsv_path = '/home/sasdemo01/data/AVI/S1/Testing/'+Father_folder+'/'
csvlist = os.listdir(mapcsv_path)

## With Side Folder or Not
With_Side_folder = True
Side = 'C'

csvlist

['19BM2025-01-00',
 '19BM2025-04-00',
 '19BM2025-05-00',
 '19BM2043-08-00',
 '19BM2044-02-00',
 '19BM2044-06-00',
 '19BM2059-06-00',
 '19BM2059-07-00',
 '19BR2031-04-00',
 '19BR2031-05-00',
 '19BM2025-01-00.csv',
 '19BM2025-04-00.csv',
 '19BM2025-05-00.csv',
 '19BM2043-08-00.csv',
 '19BM2044-02-00.csv',
 '19BM2044-06-00.csv',
 '19BM2059-06-00.csv',
 '19BM2059-07-00.csv',
 '19BR2031-04-00.csv',
 '19BR2031-05-00.csv',
 '19BC2038-04-00.csv',
 '19BC2038-04-00']

In [16]:
## Set the numerical parameters

### Scale => um/pixel
scale = 4

## X_offset => mm
## Y_offset => mm
X_offset = 0
Y_offset = 0

### CAM size(1 Unit) => mm
camsize = 16.509


### Rotate CAM
cam = np.array(cam.rotate(180))
max_y = cam.shape[0]
max_x = cam.shape[1]



## Convert and Save Image

In [18]:
if not os.path.exists(Converted_path):
    os.mkdir(Converted_path)
for file in csvlist:
    if(file.endswith('csv')):
        print(file)
        mapcsv=pd.read_csv(mapcsv_path+file)
        
        ## Convert all image
        path_prefix = Converted_path+file[:-4]+'/'
        
        if(With_Side_folder==True):
            aviimagedir = Img_path+file[:-4]+'/'+Side+'面/'
        else:
            aviimagedir = Img_path+file[:-4]+'/'
        if not os.path.exists(path_prefix):
            os.mkdir(path_prefix)
            os.mkdir(path_prefix+'NG/')
            os.mkdir(path_prefix+'OK/')
        li = (range(mapcsv.shape[0]-1))
        for n in li:
            if(n%100==0):
                print("Image: ",n," / ",mapcsv.shape[0]-1)
                print(datetime.now())
            fold = 'NG'

            im = cv.imread(aviimagedir+"NG/" + mapcsv.image_path[n])
            if(type(im)==type(None)):
                fold='OK'
                im = cv.imread(aviimagedir+"OK/" + mapcsv.image_path[n])
            ## Resize the avi image
            size1 = int(max_x*(scale*im.shape[1]/1000)/(camsize))
            size2 = int(max_y*(scale*im.shape[0]/1000)/(camsize))
            im = cv.resize(im,(size1,size2),interpolation = cv.INTER_CUBIC)

            ## defect (x,y) on cam file
            y_o = int(((mapcsv.Unit_Yvalue[n]-Y_offset)/camsize)*max_y)
            x_o = int(((mapcsv.Unit_Xvalue[n]-X_offset)/camsize)*max_x)

            ## Crop the cam image
            cam_im_1x_lefttop= (cam[y_o:y_o+int(size2),x_o:x_o+int(size1)])
            cam_im_15x_lefttop= (cam[max(y_o-int(size2*0.25),0):min(max_y,y_o+int(size2*1.25)),max(0,x_o-int(size1*0.25)):min(max_x,x_o+int(size1*1.25))])
            


            ## Preprocess image
            img = []
            img.append(im[:,:,0])
            img.append(BW_1(im[:,:,0]))
            #img.append(BW_2(im))
            img.append(Grey_1(im[:,:,0]))
            img.append(Grey_2(im[:,:,0]))
            img.append(Grey_3(im[:,:,0]))
            ## Select which size to match
            cam_to_match = cam_im_15x_lefttop.copy()
            
            ## Match Template
            res = []
            pos = []
            sim = []
            Flag = True
            for i in range(len(img)):
                try:
                    res.append(cv.matchTemplate(img[i], np.array(cam_to_match), cv.TM_CCOEFF_NORMED))
                    pos.append(np.where(res[i]>=res[i].max()))
                    sim.append(res[i].max())
                    Flag=True
                except:
                    Flag=False
            if(Flag):
                loc_most_alike = pos[np.argmax(sim)]
                index = np.argmax(sim)

                x_min = loc_most_alike[1][0].item()
                y_min = loc_most_alike[0][0].item()
                img_targ = np.array(cam_to_match)[y_min:y_min+im.shape[0],x_min:x_min+im.shape[1]]
                if(img_targ.shape!=im[:,:,0].shape):
                    img_targ=im[:,:,0].copy()
            else:
                print(n)
                img_targ = im[:,:,0].copy()
                index = 0
            im_new = im.copy()

            im_twoch = im_new.copy()
            im_twoch[:,:,2] = np.zeros([im.shape[0],im.shape[1]])
            im_twoch[:,:,1] = img_targ*0.5
            
            Image.fromarray(im_twoch).save(path_prefix + fold+'/'+mapcsv.image_path[n])        

19BM2025-01-00.csv
Image:  0  /  3670
2020-02-12 15:50:14.927320
Image:  100  /  3670
2020-02-12 15:50:23.912098
Image:  200  /  3670
2020-02-12 15:50:32.624662
Image:  300  /  3670
2020-02-12 15:50:41.174009
Image:  400  /  3670
2020-02-12 15:50:50.998701
Image:  500  /  3670
2020-02-12 15:50:59.520675
Image:  600  /  3670
2020-02-12 15:51:08.498666
Image:  700  /  3670
2020-02-12 15:51:17.207090
Image:  800  /  3670
2020-02-12 15:51:26.118281
Image:  900  /  3670
2020-02-12 15:51:34.704541
Image:  1000  /  3670
2020-02-12 15:51:43.005067
Image:  1100  /  3670
2020-02-12 15:51:52.127551
Image:  1200  /  3670
2020-02-12 15:52:01.242530
Image:  1300  /  3670
2020-02-12 15:52:10.715967
Image:  1400  /  3670
2020-02-12 15:52:19.740605
Image:  1500  /  3670
2020-02-12 15:52:28.640215
Image:  1600  /  3670
2020-02-12 15:52:37.596801
Image:  1700  /  3670
2020-02-12 15:52:46.329530
Image:  1800  /  3670
2020-02-12 15:52:55.356951
Image:  1900  /  3670
2020-02-12 15:53:04.764336
Image:  2000 

19BM2043-08-00.csv
Image:  0  /  4676
2020-02-12 16:17:52.858640
Image:  100  /  4676
2020-02-12 16:18:01.642372
Image:  200  /  4676
2020-02-12 16:18:10.920528
Image:  300  /  4676
2020-02-12 16:18:19.824644
Image:  400  /  4676
2020-02-12 16:18:28.321953
Image:  500  /  4676
2020-02-12 16:18:37.151486
Image:  600  /  4676
2020-02-12 16:18:46.715139
Image:  700  /  4676
2020-02-12 16:18:55.391844
Image:  800  /  4676
2020-02-12 16:19:06.340209
Image:  900  /  4676
2020-02-12 16:19:15.488369
Image:  1000  /  4676
2020-02-12 16:19:23.902590
Image:  1100  /  4676
2020-02-12 16:19:34.782386
Image:  1200  /  4676
2020-02-12 16:19:44.366304
Image:  1300  /  4676
2020-02-12 16:19:53.670977
Image:  1400  /  4676
2020-02-12 16:20:03.116751
Image:  1500  /  4676
2020-02-12 16:20:11.749622
Image:  1600  /  4676
2020-02-12 16:20:20.247674
Image:  1700  /  4676
2020-02-12 16:20:29.648652
Image:  1800  /  4676
2020-02-12 16:20:39.454410
Image:  1900  /  4676
2020-02-12 16:20:49.393289
Image:  2000 

Image:  5900  /  6050
2020-02-12 16:43:18.886798
Image:  6000  /  6050
2020-02-12 16:43:27.384624
19BM2059-06-00.csv
Image:  0  /  6718
2020-02-12 16:43:32.000531
Image:  100  /  6718
2020-02-12 16:43:44.477961
Image:  200  /  6718
2020-02-12 16:43:56.696132
Image:  300  /  6718
2020-02-12 16:44:08.715356
Image:  400  /  6718
2020-02-12 16:44:19.923231
Image:  500  /  6718
2020-02-12 16:44:31.653301
Image:  600  /  6718
2020-02-12 16:44:42.928660
Image:  700  /  6718
2020-02-12 16:44:54.274605
Image:  800  /  6718
2020-02-12 16:45:04.874044
Image:  900  /  6718
2020-02-12 16:45:16.217255
Image:  1000  /  6718
2020-02-12 16:45:28.326781
Image:  1100  /  6718
2020-02-12 16:45:40.487043
Image:  1200  /  6718
2020-02-12 16:45:52.441003
Image:  1300  /  6718
2020-02-12 16:46:02.785378
Image:  1400  /  6718
2020-02-12 16:46:13.837047
Image:  1500  /  6718
2020-02-12 16:46:25.280938
Image:  1600  /  6718
2020-02-12 16:46:37.889784
Image:  1700  /  6718
2020-02-12 16:46:49.689778
Image:  1800 

Image:  5900  /  9438
2020-02-12 17:13:33.733360
Image:  6000  /  9438
2020-02-12 17:13:43.715523
Image:  6100  /  9438
2020-02-12 17:13:54.724169
Image:  6200  /  9438
2020-02-12 17:14:04.340945
Image:  6300  /  9438
2020-02-12 17:14:15.573577
Image:  6400  /  9438
2020-02-12 17:14:25.871553
Image:  6500  /  9438
2020-02-12 17:14:37.552063
Image:  6600  /  9438
2020-02-12 17:14:48.321511
Image:  6700  /  9438
2020-02-12 17:14:57.972235
Image:  6800  /  9438
2020-02-12 17:15:09.071056
Image:  6900  /  9438
2020-02-12 17:15:17.672121
Image:  7000  /  9438
2020-02-12 17:15:29.285268
Image:  7100  /  9438
2020-02-12 17:15:39.462411
Image:  7200  /  9438
2020-02-12 17:15:50.772916
Image:  7300  /  9438
2020-02-12 17:16:02.608065
Image:  7400  /  9438
2020-02-12 17:16:13.455861
Image:  7500  /  9438
2020-02-12 17:16:25.773309
Image:  7600  /  9438
2020-02-12 17:16:36.089418
Image:  7700  /  9438
2020-02-12 17:16:45.819302
Image:  7800  /  9438
2020-02-12 17:16:56.951775
Image:  7900  /  943